In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q google-cloud-storage==1.43.0

In [4]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

In [ ]:
!./ngrok authtoken 233c04UxgXfFkIFFZUU5A8QpXTn_6xrgSm3QkkaURazMy8bsy

In [ ]:
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

In [ ]:
import search_frontend as se

In [ ]:
import importlib
importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

In [17]:
import json
with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [16]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
#url = 'http://516f-35-230-81-6.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      print(ap)
  except:
    pass
  
  qs_res.append((q, duration, ap))
